# Triage MIMIC - Emergency Department

This analysis relies on the emergency data from the MIMIC IV dataset (Refer to https://physionet.org/content/mimic-iv-ed/1.0/ for the original dataset.) 

First, you need to download the data from Physionet website, following the instructions on the website.

```
wget -r -N -c -np --user USERNAME --ask-password https://physionet.org/files/mimic-iv-ed/1.0/  
wget -r -N -c -np --user USERNAME --ask-password https://physionet.org/files/mimiciv/1.0/core/
```

This will result in a `physionet.org` folder in which the `ed` directory will contains all relevant data.

In [ ]:
path = 'physionet.org/files/'

##### Extract data of interest

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os

In [ ]:
# Open data
demo = pd.read_csv(os.path.join(path, 'mimiciv/1.0/core/patients.csv.gz'), index_col = 0)
triage = pd.read_csv(os.path.join(path, 'mimic-iv-ed/1.0/ed/triage.csv.gz'), index_col = [0, 1])
ed = pd.read_csv(os.path.join(path, 'mimic-iv-ed/1.0/ed/edstays.csv.gz'), index_col = [0, 2], parse_dates = ['intime', 'outtime'])

In [ ]:
# Remove unnecessary columns and datapoints with any missing data
triage = triage.drop(columns = 'chiefcomplaint')
triage = triage.dropna(0, 'any')
triage

In [ ]:
# Nurse assignment
# Expertise and tiredness might play a role here and we assign the day of admission as proxies of these dimensions
triage['nurse'] = ed.intime.dt.day_of_week[triage.index]

In [ ]:
# Acuity binarization - D
# Human decision
triage['D'] = triage['acuity'] <= 2

In [ ]:
# Outcome - Y1
# Defined as admission to the hospital
triage['Y1'] = ed.hadm_id.isna()[triage.index]

In [ ]:
# Outcome - Y2
# Defined as abnormal vital signs using Emergency Severity Index
triage['Y2'] = (triage.o2sat < 92) | (triage.resprate > 20) | (triage.heartrate > 100)

In [ ]:
# Concept - Yc
# Yc is definied as the union of Y1 and Y2
triage['YC'] = triage['Y1'] | triage['Y2']

In [ ]:
# Normalize data
triage.iloc[:, :-5] = StandardScaler().fit_transform(triage.iloc[:, :-5])

In [ ]:
triage.to_csv('triage_clean.csv')

### Verification

We study what proportion of the population have these characteristics.

In [ ]:
# Nurse assignment
triage['nurse'].value_counts().sort_index() / len(triage)

In [ ]:
# Human decision D - Acuity
triage['D'].mean()

In [ ]:
# Outcome - Y1
triage['Y1'].mean()

In [ ]:
# Outcome - Y2
triage['Y2'].mean()

In [ ]:
# Concept - Yc
triage['YC'].mean()

In [ ]:
# Intersection Y1 and Y2
(triage['Y1'] & triage['Y2']).sum() / min(triage['Y1'].sum(), triage['Y2'].sum())

In [ ]:
# Intersection Y1 concept
(triage['Y1'] & triage['YC']).sum() / triage['YC'].sum()

In [ ]:
# Intersection Y2 concept
(triage['Y2'] & triage['YC']).sum() / triage['YC'].sum()

In [ ]:
# Intersection D concept
(triage['D'] & triage['YC']).sum() / triage['YC'].sum()

In [ ]:
# Intersection D concept
(triage['D'] & triage['Y1']).sum() / min(triage['Y1'].sum(), triage['D'].sum())

In [ ]:
# Intersection D concept
(triage['D'] & triage['Y2']).sum() / min(triage['Y2'].sum(), triage['D'].sum())

----------

# Semi - synthetic labels for scenarios

We create semi synthetic labels using tree-based models to allow more control on the consistency scenarios

In [ ]:
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [ ]:
# To exhibit the desired properties in the synthetic dataset 
## Have more (but random) nurses
triage['nurse'] = np.random.choice(np.arange(20), size = len(triage))

**Scenario 1**: One model for all experts and randomness in high consistency for Y1 (experts agree on Y2 and might therefore benefit YC modelling)

In [ ]:
# Model for Y1
model_y1 = DecisionTreeClassifier(max_depth = 15, random_state = 42)
model_y1.fit(triage.iloc[:, :7], triage['Y1'])
synth_y1 = model_y1.predict_proba(triage.iloc[:, :7])[:, 1]
roc_auc_score(triage['Y1'], synth_y1)

In [ ]:
# Model for Y2
model_y2 = DecisionTreeClassifier(max_depth = 2, random_state = 42)
model_y2.fit(triage.iloc[:, :7], triage['Y2'])
synth_y2 = model_y2.predict_proba(triage.iloc[:, :7])[:, 1]
roc_auc_score(triage['Y2'], synth_y2)

In [ ]:
# Update labels
triage['Y1'] = synth_y1 > 0.5
triage['Y2'] = synth_y2 > 0.5
triage['YC'] = triage['Y1'] | triage['Y2']

In [ ]:
# Model for D : Use a model for Yc and chance some of the leaved decision
model_yc = DecisionTreeClassifier(max_depth = 4, random_state = 42)
model_yc.fit(triage.iloc[:, :7], triage['YC'])
synth_yc = model_yc.predict_proba(triage.iloc[:, :7])[:, 1]
roc_auc_score(triage['YC'], synth_yc)

In [ ]:
# Compute last leaves of each point
final_leave_yc = model_yc.apply(triage.iloc[:, :7])

# Compute precision in Y2 for each leave
for leaf in np.unique(final_leave_yc):
    selection = final_leave_yc == leaf
    print('\n{} -> {} patients ({:.2f} % population)'.format(leaf, selection.sum(), 100*selection.mean()))
    print('Y1 -> {:.2f} precision - {:.2f} % patients'.format(precision_score(triage['Y1'][selection], synth_yc[selection] > 0.5, zero_division = 0), 100*triage['Y1'][selection].mean()))
    print('Y2 -> {:.2f} precision - {:.2f} % patients'.format(precision_score(triage['Y2'][selection], synth_yc[selection] > 0.5, zero_division = 0), 100*triage['Y2'][selection].mean()))

In [ ]:
# Randomly change prediction for leaves with high precision for Y1
leaves_to_update = [8, 16, 20, 21, 23, 24]
synth_yc_sc1 = synth_yc.copy()

noise = np.random.uniform(size = len(final_leave_yc)) > 0.3
for leaf in leaves_to_update:
    selection = final_leave_yc == leaf
    synth_yc_sc1[selection & noise & triage.Y1] = 0
    synth_yc_sc1[selection & noise & ~triage.Y1] = 1
    print(leaf, np.mean(synth_yc_sc1[selection] > 0.5))

In [ ]:
triage['D'] = synth_yc_sc1 > 0.5

In [ ]:
triage.to_csv('triage_scenario_1.csv')

**Scenario 2**: Non random assignment with bias. Older patients are assigned to one expert who is biased by underestimating their risk (D == 0).

In [ ]:
triages2 = triage.copy()
triages2['D'] = triages2['YC'] # Initialize close to oracle

In [ ]:
age = triages2.join(demo).anchor_age
triages2.loc[age > 70, 'nurse'] = 15 # Non random assignment with new nurse
triages2.loc[age > 70, 'D'] = 0 

In [ ]:
triages2.to_csv('triage_scenario_2.csv')

**Scenario 4**: Noise dependent on experts. Different experts come with different expertise. We model this with expert-specific noise.

In [ ]:
proba_error = np.random.uniform(size = len(np.unique(triage.nurse)))
noises = {nurse: np.random.uniform(size = len(triage)) > proba_error[nurse] for nurse in np.unique(triage.nurse)}

In [ ]:
triages4 = triage.copy()

In [ ]:
# Randomly change prediction for leaves with high precision for Y1
synth_yc_sc4 = synth_yc.copy()

for leaf in leaves_to_update:
    selection = final_leave_yc == leaf

    for nurse in noises:
        selection_nurse = selection & noises[nurse] & (triage.nurse == nurse)
        synth_yc_sc4[selection_nurse & triage.Y1] = 0
        synth_yc_sc4[selection_nurse & ~triage.Y1] = 1
        print(leaf, nurse, np.mean(synth_yc_sc4[selection_nurse] > 0.5))
    print(leaf, np.mean(synth_yc_sc4[selection] > 0.5), '\n')

In [ ]:
triage['D'] = synth_yc_sc4 > 0.5

In [ ]:
triages4.to_csv('triage_scenario_4.csv')